### 引入相關python模組

In [ ]:
from mycnn import UNet
from mycnn import utils
import tensorflow as tf
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from mycnn import data
data.generate_classification_dataset(
    r'D:\Datasets\DogsVsCats\train',
    shuffle=True,
    gray=True,
    validation_split=0.2
)

### 載入UNet模型

In [ ]:
unet = UNet(classes_num=1)
unet.summary()

In [ ]:
def parse_fn(dataset, **kwargs):
    # 分離 dataset
    path_x = dataset["in"]
    path_y = dataset["out"]

    # 讀取 image
    x = tf.io.read_file(path_x)
    x = tf.io.decode_image(x, channels=1, expand_animations=False)
    y = tf.io.read_file(path_y)
    y = tf.io.decode_image(y, channels=1, expand_animations=False)

    # 對影像進行正規化，及增加影像通道
    x = tf.cast(x, tf.float32) / 255.0
    x = tf.expand_dims(x, axis=-1)
    y = tf.cast(y, tf.float32) / 255.0
    y = tf.expand_dims(y, axis=-1)

    return x, y # 回傳資料

In [ ]:
# tra_data_in = glob.glob(r"data\in\*.png")
# tra_data_out = glob.glob(r"data\out\*.png")

tra_data_in = ["2007_001430.jpg"]
tra_data_out = ["2007_001430.png"]

tra_data = {}
tra_data["in"] = tra_data_in
tra_data["out"] = tra_data_out

autotune = tf.data.experimental.AUTOTUNE
#-----------------------------------------------------------------------------#
tra_ds = tf.data.Dataset.from_tensor_slices(tra_data)
tra_ds = tra_ds.map(lambda ds: parse_fn(ds), num_parallel_calls=autotune)
tra_ds = tra_ds.batch(1)

In [ ]:
unet.compile(optimizer=tf.keras.optimizers.Adam(),
             loss="mse")
unet.fit(tra_ds)